In [2]:
import pandas as pd
import os
import os.path as osp

In [3]:
dataset_abs_path = '/Users/myZZUNG/myworkspace/posco_ai_challenge/dataset/1. 도전과제/'
dataset_name = '[과제3] 식당 메뉴 및 식사인원.xlsx'

In [4]:
xl = pd.ExcelFile(osp.join(dataset_abs_path, dataset_name))

In [5]:
xl.sheet_names

['설명', '식사인원', '메뉴']

In [6]:
rest_num = xl.parse('식사인원')
rest_num.info()
rest_num.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33282 entries, 1 to 33282
Data columns (total 3 columns):
매출일자        33282 non-null int64
품명          33282 non-null object
수량          32248 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.0+ MB


,매출일자    ,품명  ,수량
1,20030301,아침식사    ,37.472924
2,20030301,점심식사    ,31.191336
3,20030301,저녁식사    ,19.566787
4,20030302,아침식사    ,36.101083
5,20030302,점심식사    ,33.357401


In [7]:
rest_num[rest_num[rest_num.columns[0]] == 20030828]

,매출일자    ,품명  ,수량
539,20030828,아침식사    ,40.722022
540,20030828,아침식사1    ,100.000000
541,20030828,점심식사    ,47.509025
542,20030828,저녁식사    ,23.176895


In [8]:
rest_num.columns

Index(['매출일자    ', '품명  ', '수량'], dtype='object')

In [9]:
rest_num[rest_num.columns[1]].value_counts()

저녁식사               5346
아침식사               5346
점심식사               5343
점심식사2(양식)          4571
아침식사1              4267
저녁식사1              4253
점심식사1              4156
Name: 품명  , dtype: int64

### 데이터 설명
* 아침 = 아침식사 + 아침식사1
* 저녁 = 저녁식사 + 저녁식사1
* 점심 = 점심 + 점심1
* 점심(양식) = 점심2

In [10]:
def parse_foodtime(x):
    if '아침' in x:
        return '아침'
    if '저녁' in x:
        return '저녁'
    if '점심식사2'in x:
        return '점심2'
    if '점심식사' in x:
        return '점심'

In [11]:
rest_num['time'] = rest_num[rest_num.columns[1]].apply(lambda x: parse_foodtime(x))

In [12]:
proc_rest_num = rest_num.groupby([rest_num.columns[0],'time']).sum().reset_index()
proc_rest_num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20606 entries, 0 to 20605
Data columns (total 3 columns):
매출일자        20606 non-null int64
time        20606 non-null object
수량          20606 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 483.0+ KB


In [13]:
menu = xl.parse('메뉴')
menu.info()
menu.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24860 entries, 1 to 24860
Data columns (total 3 columns):
일자      24860 non-null int64
식사명     24860 non-null object
식사내용    24860 non-null object
dtypes: int64(1), object(2)
memory usage: 776.9+ KB


,일자,식사명,식사내용
1,20030301,아침식사,"과일샐러드,닭죽,돈육마늘장조림,떡만두국,부추김무침,쌀밥"
2,20030301,아침식사2,"딸기잼(중),비엔나구이,스크램블에그(경양식),야채샐러드,크림스프(경양식),토스트&모닝빵"
3,20030301,점심식사,"골뱅이야채무침,새우맛살튀김,쌀밥(사무직),열무겉절이,칼국수"
4,20030301,저녁식사,"감자으깸샐러드,비프까스,스위트피클,쌀밥,옥수수스프"
5,20030302,아침식사,"계란죽,곤약멸치조림,김치국,마카로니샐러드,쌀밥,오징어회무침"


In [14]:
menu['식사명'].value_counts()

저녁식사     5367
아침식사     5367
점심식사     5365
점심식사2    4571
아침식사2    4190
Name: 식사명, dtype: int64

In [15]:
menu['time'] = menu['식사명'].apply(lambda x: parse_foodtime(x))
menu['식사내용'] = menu['식사내용'].apply(lambda x: x+',')

In [16]:
proc_menu = menu.groupby(['일자','time']).sum().drop(['식사명'],axis=1).reset_index()
proc_menu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20670 entries, 0 to 20669
Data columns (total 3 columns):
일자      20670 non-null int64
time    20670 non-null object
식사내용    20670 non-null object
dtypes: int64(1), object(2)
memory usage: 484.5+ KB


In [17]:
proc_menu

,일자,time,식사내용
0,20030301,아침,"과일샐러드,닭죽,돈육마늘장조림,떡만두국,부추김무침,쌀밥,딸기잼(중),비엔나구이,스크..."
1,20030301,저녁,"감자으깸샐러드,비프까스,스위트피클,쌀밥,옥수수스프,"
2,20030301,점심,"골뱅이야채무침,새우맛살튀김,쌀밥(사무직),열무겉절이,칼국수,"
3,20030302,아침,"계란죽,곤약멸치조림,김치국,마카로니샐러드,쌀밥,오징어회무침,딸기잼(중),삶은계란,야..."
4,20030302,저녁,"계란탕,단무지잔파무침,자장소스,잡채밥,탕수만두,"
5,20030302,점심,"갈비탕,깍두기(손칼),두부달걀전,생선묵조림,쌀밥,오이,풋고추,쌈장,"
6,20030303,아침,"감자다시마국,녹두죽,멸치볶음,쇠고기팽이버섯볶음,쌀밥,우유,호박나물,딸기&쵸코우유,딸..."
7,20030303,저녁,"꽈리고추찜,돈등심케찹조림,쌀밥,어묵양파볶음(K),열무된장무침,콩나물무국,"
8,20030303,점심,"계란찜,냉이배추된장국,삼치엿장조림,시금치나물,쌀밥,콩나물무침,"
9,20030304,아침,"감자채볶음,고구마죽,동태무조림(가공),두부유부된장국,부추김무침,쌀밥,우유1,딸기&쵸..."


In [18]:
merged_df = proc_menu.merge(proc_rest_num, left_on=['일자','time'], right_on=[proc_rest_num.columns[0],'time'],how='outer')
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20670 entries, 0 to 20669
Data columns (total 5 columns):
일자          20670 non-null int64
time        20670 non-null object
식사내용        20670 non-null object
매출일자        20606 non-null float64
수량          20606 non-null float64
dtypes: float64(2), int64(1), object(2)
memory usage: 968.9+ KB


In [19]:
merged_df[merged_df[merged_df.columns[3]].isnull()]

,일자,time,식사내용,매출일자    ,수량
1089,20040315,아침,"돈육장조림,멸치볶음,시금치나물,쌀밥,쑥배추된장국,야채죽,우유,포기김치,딸기잼(대),...",NaN,NaN
1090,20040315,저녁,"미나리나물,쇠고기야채볶음,쌀밥,오징어조림,콩가루배추국,콩나물무침,포기김치,",NaN,NaN
1091,20040315,점심,"갈치그릴구이,돈등뼈감자탕,메밀묵상추무침,사과,석박지,쌀밥,천사채샐러드,",NaN,NaN
1092,20040316,아침,"누룽지,두부달걀전,쇠고기무국,쌀밥,열기찜,열무된장무침,우유,포기김치,당근스프(경양식...",NaN,NaN
1093,20040316,저녁,"건취나물볶음,골뱅이야채무침,꽈리고추찜,대구지리(가공),돈육두루치기,쌀밥,포기김치,",NaN,NaN
1094,20040316,점심,"닭갈비,상추깻잎쌈,생선까스,쌀밥,아욱국,요구르트,포기김치,풋마늘생채,",NaN,NaN
1095,20040317,아침,"가자미양념장구이,감자베이컨볶음,두부알탕,쌀밥,우유,포기김치,표고피망볶음,흑임자죽,감...",NaN,NaN
1096,20040317,저녁,"김치전,꽁치무조림,쌀밥,어묵고추장볶음,육개장,통도라지무침,포기김치,",NaN,NaN
1097,20040317,점심,"미역국,삼치엿장조림,새우살부추냉채,쇠고기팽이버섯볶음,식혜,쌀밥,참나물겉절이,포기김치,",NaN,NaN
1098,20040318,아침,"건새우호박볶음,계란찜,근대국,쌀밥,우유,조기양념장구이,콩죽,포기김치,딸기잼(대),모...",NaN,NaN


In [20]:
train_df = merged_df.dropna().copy()

In [21]:
train_df.to_csv('train_data.tsv',sep='\t',index=False)